# Dynamics 365 Business Central Trouble Shooting Guide (TSG) - Performance analysis (partner code issues)

This notebook contains Kusto queries that can help getting to the root cause of a performance issue for an environment.

The notebook highlights issues for object ids in the ranges defined for partner code \[50000..99999\] and \[1 mio..75 mio\] and can be used to find performance problems in code in per-tenant extensions or app source extensions.

Each section in the notebook contains links to the performance tuning guide on docs [aka.ms/bcperformance](aka.ms\bcperformance), links to the documentation of relevant telemetry in [aka.ms/bctelemetry](aka.ms\bctelemetry), as well as Kusto queries that help dive into a specific area (sessions, web service requests, database calls, reports, and page load times).

NB! Some of the signal used in this notebook is only available in newer versions of Business Central, so check the version of your environment if some sections do not return any data. The signal documentation states in which version a given signal was introduced.

## 1. Connect to Application Insights
First you need to set the notebook Kernel to Python3, load the KQLmagic module (did you install it?) and connect to your Application Insights resource (get appid and appkey from the API access page in the Application Insights portal)

In [23]:
# load the KQLmagic module
%reload_ext Kqlmagic

In [24]:
# Connect to the Application Insights API
%kql appinsights://appid='<add Application ID id from the Application Insights portal (under API access)>';appkey='<add an API key (generate one from the Application Insights portal (under API access))>'


## 2. Define filters
This workbook is designed for troubleshooting a single environment. Please provide values for aadTenantId and environmentName: 

In [25]:
#aadTenantId = "<aadTenantId>"
#environmentName = "<environmentName>"

# date filters for the analysis
# use YYYY-MM-DD format for the dates (ISO 8601)
startDate = "2020-11-20"
endDate = "2020-11-30"

# Analyze performance
Now you can run Kusto queries to look for possible root causes for performance issues.

Either click **Run All** above to run all sections, or scroll down to the type of analysis you want to do and manually run queries

## Web service requests

Performance tuning guide: https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/performance/performance-developer#writing-efficient-web-services

Web service telemetry docs: https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/administration/telemetry-webservices-trace

KQL samples: https://github.com/microsoft/BCTech/blob/master/samples/AppInsights/KQL/RawData/WebServiceCalls.kql

In [26]:
%%kql
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
let _startDate = startDate;
let _endDate = endDate;
traces
| where 1==1 
    and timestamp >= todatetime(_startDate)
    and timestamp <= todatetime(_endDate) + totimespan(24h) - totimespan(1ms)   
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.environmentName == _environmentName
    and customDimensions.eventId == 'RT0006'
| extend alObjectId = toint(customDimensions.alObjectId)
| where alObjectId between (50000 .. 99999) or alObjectId between (1000000 .. 74999999)
| extend category = tostring( customDimensions.category )
| extend codeOwnership = case(
      alObjectId between (50000 .. 99999), 'ENVIRONMENT'
    , alObjectId between (1000000 .. 69999999), 'ISV_ONPREM'
    , alObjectId between (70000000 .. 74999999), 'ISV_ONLINE'
    , 'MICROSOFT'
)
| summarize request_count=count() by category, bin(timestamp, 1d)
| render timechart title= 'Number of web service requests by category endpoints (in partner code)'

In [27]:
%%kql
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
let _startDate = startDate;
let _endDate = endDate;
traces
| where 1==1 
    and timestamp >= todatetime(_startDate)
    and timestamp <= todatetime(_endDate) + totimespan(24h) - totimespan(1ms)   
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.environmentName == _environmentName
    and customDimensions.eventId == 'RT0008'
| extend alObjectId = toint(customDimensions.alObjectId)
| where alObjectId between (50000 .. 99999) or alObjectId between (1000000 .. 74999999)
| extend category = tostring( customDimensions.category )
       , executionTimeInMS = toreal(totimespan(customDimensions.serverExecutionTime))/10000 //the datatype for executionTime is timespan 
| extend codeOwnership = case(
      alObjectId between (50000 .. 99999), 'ENVIRONMENT'
    , alObjectId between (1000000 .. 69999999), 'ISV_ONPREM'
    , alObjectId between (70000000 .. 74999999), 'ISV_ONLINE'
    , 'MICROSOFT'
)
| where executionTimeInMS <= 10000 // only include ws calls with execution time <= 10 sec
| summarize count() by executionTime_ms = bin(executionTimeInMS, 100), category
| extend log_count = log10(count_)
| order by category, executionTime_ms asc
| render columnchart with  (ycolumns = log_count, ytitle='Log10(count)', series = category, title= 'Execution time (in ms) of normal web service requests by category (in partner code)' ) 

In [28]:
%%kql
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
let _startDate = startDate;
let _endDate = endDate;
traces
| where 1==1 
    and timestamp >= todatetime(_startDate)
    and timestamp <= todatetime(_endDate) + totimespan(24h) - totimespan(1ms)   
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.environmentName == _environmentName
    and customDimensions.eventId == 'RT0008'
| extend alObjectId = toint(customDimensions.alObjectId)
| where alObjectId between (50000 .. 99999) or alObjectId between (1000000 .. 74999999)
| extend category = tostring( customDimensions.category )
       , executionTimeInSec = toreal(totimespan(customDimensions.serverExecutionTime))/10000 /1000 //the datatype for executionTime is timespan 
| extend codeOwnership = case(
      alObjectId between (50000 .. 99999), 'ENVIRONMENT'
    , alObjectId between (1000000 .. 69999999), 'ISV_ONPREM'
    , alObjectId between (70000000 .. 74999999), 'ISV_ONLINE'
    , 'MICROSOFT'
)
| where executionTimeInSec > 10 // only include ws calls with execution time > 10 sec
| summarize count() by executionTime_sec = bin(executionTimeInSec, 10), category
| extend log_count = log10(count_)
| order by category, executionTime_sec asc
| render columnchart with  (ycolumns = log_count, ytitle='log10(count)', series = category, title= 'Execution time (in sec) of slow web service requests by category (in partner code)' ) 

In [29]:
%%kql
// 
// top 20 most expensive web service calls (partner code)
// 
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
let _startDate = startDate;
let _endDate = endDate;
traces
| where 1==1 
    and timestamp >= todatetime(_startDate)
    and timestamp <= todatetime(_endDate) + totimespan(24h) - totimespan(1ms)   
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.environmentName == _environmentName
    and customDimensions.eventId == 'RT0008'
| extend alObjectId = toint(customDimensions.alObjectId)
| where alObjectId between (50000 .. 99999) or alObjectId between (1000000 .. 74999999)
| extend category = tostring( customDimensions.category )
       , endpoint = tostring( customDimensions.endpoint )
       , executionTimeInMS = toreal(totimespan(customDimensions.serverExecutionTime))/10000 //the datatype for executionTime is timespan 
| extend codeOwnership = case(
      alObjectId between (50000 .. 99999), 'ENVIRONMENT'
    , alObjectId between (1000000 .. 69999999), 'ISV_ONPREM'
    , alObjectId between (70000000 .. 74999999), 'ISV_ONLINE'
    , 'MICROSOFT'
)
| summarize count=count(), medianExecutionTimeInMS=percentile(executionTimeInMS, 50) by alObjectId, category, endpoint
| order by medianExecutionTimeInMS desc
| limit 20

## (Outgoing) web service requests overview 

Performance tuning guide: [https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/performance/performance-developer#web-service-client-performance](https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/performance/performance-developer#writing-efficient-web-services)

Outgoing Web service telemetry docs: [https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/administration/telemetry-webservices-outgoing-trace](https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/administration/telemetry-webservices-outgoing-trace) 

KQL sample: [https://github.com/microsoft/BCTech/blob/master/samples/AppInsights/KQL/RawData/OutGoingWebServiceCalls.kql](https://github.com/microsoft/BCTech/blob/master/samples/AppInsights/KQL/RawData/OutGoingWebServiceCalls.kql)

Explanation of different HTTP status codes: [https://en.wikipedia.org/wiki/List_of_HTTP_status_codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes)

In [30]:
%%kql
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
let _startDate = startDate;
let _endDate = endDate;
traces
| where 1==1 
    and timestamp >= todatetime(_startDate)
    and timestamp <= todatetime(_endDate) + totimespan(24h) - totimespan(1ms)    
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.environmentName == _environmentName
    and customDimensions.eventId == 'RT0019'
| extend alObjectId = toint(customDimensions.alObjectId)    
| where alObjectId between (50000 .. 99999) or alObjectId between (1000000 .. 74999999)
| extend httpStatusCode = tostring( customDimensions.httpReturnCode )
| extend codeOwnership = case(
      alObjectId between (50000 .. 99999), 'ENVIRONMENT'
    , alObjectId between (1000000 .. 69999999), 'ISV_ONPREM'
    , alObjectId between (70000000 .. 74999999), 'ISV_ONLINE'
    , 'MICROSOFT'
)
| summarize request_count=count() by httpStatusCode, bin(timestamp, 1d)
| order by httpStatusCode asc
| render timechart title= 'Number of outgoing web service requests by HTTP status code (partner code)'

In [31]:
%%kql
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
let _startDate = startDate;
let _endDate = endDate;
traces
| where 1==1 
    and timestamp >= todatetime(_startDate)
    and timestamp <= todatetime(_endDate) + totimespan(24h) - totimespan(1ms)        
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.environmentName == _environmentName
    and customDimensions.eventId == 'RT0019'
| extend httpMethod = tostring( toupper( customDimensions.httpMethod ) )
       , executionTimeInSec = toreal(totimespan(customDimensions.serverExecutionTime))/10000 /1000 //the datatype for executionTime is timespan 
| extend alObjectId = toint(customDimensions.alObjectId)    
| where alObjectId between (50000 .. 99999) or alObjectId between (1000000 .. 74999999)
| extend httpStatusCode = tostring( customDimensions.httpReturnCode )
| extend codeOwnership = case(
      alObjectId between (50000 .. 99999), 'ENVIRONMENT'
    , alObjectId between (1000000 .. 69999999), 'ISV_ONPREM'
    , alObjectId between (70000000 .. 74999999), 'ISV_ONLINE'
    , 'MICROSOFT'
)
| where executionTimeInSec <= 10
| summarize count() by executionTime_sec = bin(executionTimeInSec, 1), httpMethod
| extend log_count = log10( count_ )
| order by httpMethod, executionTime_sec asc
| render columnchart with  (ycolumns = log_count, ytitle='log(count)', series = httpMethod, title= 'Execution time (seconds) of "normal" outgoing ws requests by method' ) 


In [32]:
%%kql
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
let _startDate = startDate;
let _endDate = endDate;
traces
| where 1==1 
    and timestamp >= todatetime(_startDate)
    and timestamp <= todatetime(_endDate) + totimespan(24h) - totimespan(1ms)        
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.environmentName == _environmentName
    and customDimensions.eventId == 'RT0019'
| extend httpMethod = tostring( toupper( customDimensions.httpMethod ) )
       , executionTimeInSec = toreal(totimespan(customDimensions.serverExecutionTime))/10000 /1000 //the datatype for executionTime is timespan 
| extend alObjectId = toint(customDimensions.alObjectId)    
| where alObjectId between (50000 .. 99999) or alObjectId between (1000000 .. 74999999)
| extend httpStatusCode = tostring( customDimensions.httpReturnCode )
| extend codeOwnership = case(
      alObjectId between (50000 .. 99999), 'ENVIRONMENT'
    , alObjectId between (1000000 .. 69999999), 'ISV_ONPREM'
    , alObjectId between (70000000 .. 74999999), 'ISV_ONLINE'
    , 'MICROSOFT'
)
| where executionTimeInSec > 10
| summarize count() by executionTime_sec = bin(executionTimeInSec, 10), httpMethod
| extend log_count = log10( count_ )
| order by httpMethod, executionTime_sec asc
| render columnchart with  (ycolumns = log_count, ytitle='log10(count)', series = httpMethod, title= 'Execution time (seconds) of slow outgoing ws requests by method' ) 


In [33]:
%%kql
//
// Top 20 most expensive outgoing calls
//
// Includes descriptive statistics avg, min, max, and 95 percentile for execution time and number of calls
//
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
let _startDate = startDate;
let _endDate = endDate;
traces
| where 1==1 
    and timestamp >= todatetime(_startDate)
    and timestamp <= todatetime(_endDate) + totimespan(24h) - totimespan(1ms)    
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.environmentName == _environmentName
    and customDimensions.eventId == 'RT0019'
| extend httpMethod = tostring( toupper( customDimensions.httpMethod ) )
       , executionTimeInMS = toreal(totimespan(customDimensions.serverExecutionTime))/10000 //the datatype for executionTime is timespan 
       , alObjectId = tostring( customDimensions.alObjectId )
       , alObjectName = tostring( customDimensions.alObjectName )
       , alObjectType = tostring( customDimensions.alObjectType )
, endpoint = tostring( customDimensions.endpoint )
, extensionId = tostring( customDimensions.extensionId )
, extensionName = tostring( customDimensions.extensionName )
, extensionVersion = tostring( customDimensions.extensionVersion )
| extend alObjectId = toint(customDimensions.alObjectId)    
| where alObjectId between (50000 .. 99999) or alObjectId between (1000000 .. 74999999)
| extend httpStatusCode = tostring( customDimensions.httpReturnCode )
| extend codeOwnership = case(
      alObjectId between (50000 .. 99999), 'ENVIRONMENT'
    , alObjectId between (1000000 .. 69999999), 'ISV_ONPREM'
    , alObjectId between (70000000 .. 74999999), 'ISV_ONLINE'
    , 'MICROSOFT'
)
| summarize avg(executionTimeInMS), min(executionTimeInMS), max(executionTimeInMS),percentile(executionTimeInMS,95), count() by 
  httpMethod
, alObjectId, alObjectName, alObjectType, codeOwnership
, extensionId, extensionName, extensionVersion
, endpoint
| order by avg_executionTimeInMS desc
| project avg_timeInSec=round(avg_executionTimeInMS/1000,2)
, min_timeInSec=round(min_executionTimeInMS/1000,2)
, max_timeInSec=round(max_executionTimeInMS/1000,2)
, perc_95InSec=round(percentile_executionTimeInMS_95/1000,2), count_
, alObjectId, alObjectName, alObjectType, codeOwnership
, extensionId, extensionName, extensionVersion, httpMethod, endpoint
| limit 20

## Data related

Performance tuning guide: 
* [Efficient data access](https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/performance/performance-developer#efficient-data-access)
* [Avoid locking](https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/performance/performance-application#avoid-locking)

Database telemetry docs: 
* https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/administration/telemetry-long-running-sql-query-trace
* https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/administration/telemetry-database-locks-trace

KQL samples:
* https://github.com/microsoft/BCTech/blob/master/samples/AppInsights/KQL/RawData/Long%20Running%20SQL%20Queries.kql
* https://github.com/microsoft/BCTech/blob/master/samples/AppInsights/KQL/RawData/LockTimeouts.kql


In [34]:
%%kql
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
let _startDate = startDate;
let _endDate = endDate;
traces
| where 1==1 
    and timestamp >= todatetime(_startDate)
    and timestamp <= todatetime(_endDate) + totimespan(24h) - totimespan(1ms)   
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.environmentName == _environmentName
    and customDimensions.eventId == 'RT0005'
| extend alObjectId = toint(customDimensions.alObjectId)
| where alObjectId between (50000 .. 99999) or alObjectId between (1000000 .. 74999999)
| extend codeOwnership = case(
      alObjectId between (50000 .. 99999), 'ENVIRONMENT'
    , alObjectId between (1000000 .. 69999999), 'ISV_ONPREM'
    , alObjectId between (70000000 .. 74999999), 'ISV_ONLINE'
    , 'MICROSOFT'
)
| summarize count() by bin(timestamp, 1d)
| render timechart title= 'Number of long running SQL queries (in partner code)'

In [35]:
%%kql
// 
// top 20 most expensive long running sql queries (in partner code)
// 
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
let _startDate = startDate;
let _endDate = endDate;
traces
| where 1==1 
    and timestamp >= todatetime(_startDate)
    and timestamp <= todatetime(_endDate) + totimespan(24h) - totimespan(1ms)   
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.environmentName == _environmentName
    and customDimensions.eventId == 'RT0005'
| extend alObjectId = toint(customDimensions.alObjectId)
       , alObjectName = toint(customDimensions.alObject)
       , executionTimeInMS = toreal(totimespan(customDimensions.executionTime))/10000 //the datatype for executionTime is timespan 
| where alObjectId between (50000 .. 99999) or alObjectId between (1000000 .. 74999999)
| extend codeOwnership = case(
      alObjectId between (50000 .. 99999), 'ENVIRONMENT'
    , alObjectId between (1000000 .. 69999999), 'ISV_ONPREM'
    , alObjectId between (70000000 .. 74999999), 'ISV_ONLINE'
    , 'MICROSOFT'
)
| summarize count(), avg(executionTimeInMS) by alObjectId
| limit 20

In [36]:
%%kql
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
let _startDate = startDate;
let _endDate = endDate;
traces
| where 1==1 
    and timestamp >= todatetime(_startDate)
    and timestamp <= todatetime(_endDate) + totimespan(24h) - totimespan(1ms)   
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.environmentName == _environmentName
    and customDimensions.eventId == 'RT0012'
| extend alObjectId = toint(customDimensions.alObjectId)
| where alObjectId between (50000 .. 99999) or alObjectId between (1000000 .. 74999999)
| extend codeOwnership = case(
      alObjectId between (50000 .. 99999), 'ENVIRONMENT'
    , alObjectId between (1000000 .. 69999999), 'ISV_ONPREM'
    , alObjectId between (70000000 .. 74999999), 'ISV_ONLINE'
    , 'MICROSOFT'
)
| summarize request_count=count() by bin(timestamp, 1d)
| render timechart title= 'Number of database lock timeouts (in partner code)'

In [37]:
%%kql
// 
// partner code units where database lock timeouts occur
// 
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
let _startDate = startDate;
let _endDate = endDate;
traces
| where 1==1 
    and timestamp >= todatetime(_startDate)
    and timestamp <= todatetime(_endDate) + totimespan(24h) - totimespan(1ms)   
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.environmentName == _environmentName
    and customDimensions.eventId == 'RT0012'
| extend alObjectId = toint(customDimensions.alObjectId)
| where alObjectId between (50000 .. 99999) or alObjectId between (1000000 .. 74999999)
| extend codeOwnership = case(
      alObjectId between (50000 .. 99999), 'ENVIRONMENT'
    , alObjectId between (1000000 .. 69999999), 'ISV_ONPREM'
    , alObjectId between (70000000 .. 74999999), 'ISV_ONLINE'
    , 'MICROSOFT'
)
| summarize timeout_count=count() by alObjectId
| order by timeout_count

## Reports

Learn more about how to write performant reports here in the performance tuning guide: https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/performance/performance-developer#writing-efficient-reports

Report telemetry docs: https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/administration/telemetry-reports-trace

KQL samples: 
* https://github.com/microsoft/BCTech/blob/master/samples/AppInsights/KQL/RawData/Reports.kql
* https://github.com/microsoft/BCTech/blob/master/samples/AppInsights/KQL/PerformanceTuning/ReportExecution.kql

In [38]:
%%kql
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
let _startDate = startDate;
let _endDate = endDate;
traces
| where 1==1 
    and timestamp >= todatetime(_startDate)
    and timestamp <= todatetime(_endDate) + totimespan(24h) - totimespan(1ms)   
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.environmentName == _environmentName
    and customDimensions.eventId == 'RT0006'
| extend alObjectId = toint(customDimensions.alObjectId)
| where alObjectId between (50000 .. 99999) or alObjectId between (1000000 .. 74999999)
| extend clientType = tostring( customDimensions.clientType )
       , reportName = tostring( customDimensions.alObjectName )
| extend codeOwnership = case(
      alObjectId between (50000 .. 99999), 'ENVIRONMENT'
    , alObjectId between (1000000 .. 69999999), 'ISV_ONPREM'
    , alObjectId between (70000000 .. 74999999), 'ISV_ONLINE'
    , 'MICROSOFT'
)
| where reportName <> ''
| summarize count=count() by clientType, bin(timestamp, 1d)
| render timechart title= 'Number of report execution by client/session type (partner reports)'

In [39]:
%%kql
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
let _startDate = startDate;
let _endDate = endDate;
traces
| where 1==1 
    and timestamp >= todatetime(_startDate)
    and timestamp <= todatetime(_endDate) + totimespan(24h) - totimespan(1ms)   
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.environmentName == _environmentName
    and customDimensions.eventId == 'RT0006'
| extend alObjectId = toint(customDimensions.alObjectId)
| where alObjectId between (50000 .. 99999) or alObjectId between (1000000 .. 74999999)
| extend reportName = tostring( customDimensions.alObjectName )
       , executionTimeInSec = toreal(totimespan(customDimensions.totalTime))/10000000 //the datatype for executionTime is timespan 
| where reportName <> ''
| extend codeOwnership = case(
      alObjectId between (50000 .. 99999), 'ENVIRONMENT'
    , alObjectId between (1000000 .. 69999999), 'ISV_ONPREM'
    , alObjectId between (70000000 .. 74999999), 'ISV_ONLINE'
    , 'MICROSOFT'
)
| summarize avg=avg(executionTimeInSec), median=percentile(executionTimeInSec, 50), percentile95=percentile(executionTimeInSec, 95), max=max(executionTimeInSec)   
  by reportName
| order by percentile95
| limit 10
| render columnchart with  (title= 'Execution time stats of (partner) reports by report name (top 10 by 95% percentile)', ytitle='Time (in seconds)' ) 

In [40]:
%%kql
// 
// top 20 most expensive reports (in partner code)
// 
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
let _startDate = startDate;
let _endDate = endDate;
traces
| where 1==1 
    and timestamp >= todatetime(_startDate)
    and timestamp <= todatetime(_endDate) + totimespan(24h) - totimespan(1ms)   
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.environmentName == _environmentName
    and customDimensions.eventId == 'RT0006'
| extend alObjectId = toint(customDimensions.alObjectId)
| where alObjectId between (50000 .. 99999) or alObjectId between (1000000 .. 74999999)
| extend reportName = tostring( customDimensions.alObjectName )
       , executionTimeInSec = toreal(totimespan(customDimensions.totalTime))/10000000 //the datatype for executionTime is timespan 
| extend codeOwnership = case(
      alObjectId between (50000 .. 99999), 'ENVIRONMENT'
    , alObjectId between (1000000 .. 69999999), 'ISV_ONPREM'
    , alObjectId between (70000000 .. 74999999), 'ISV_ONLINE'
    , 'MICROSOFT'
)
| where reportName <> ''
| summarize avg=avg(executionTimeInSec), median=percentile(executionTimeInSec, 50), percentile95=percentile(executionTimeInSec, 95), max=max(executionTimeInSec)   
  by reportName
| order by percentile95
| limit 20

In [41]:
%%kql
// 
// top 10 reports that have a low sqlratio
// 
// The higher the number called sqlratio is, the more set-based the report is (it tells you how many sql rows are read per sql statement)
//
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
let _startDate = startDate;
let _endDate = endDate;
traces
| where 1==1 
    and timestamp >= todatetime(_startDate)
    and timestamp <= todatetime(_endDate) + totimespan(24h) - totimespan(1ms)   
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.environmentName == _environmentName
    and customDimensions.eventId == 'RT0006'
| extend alObjectId = toint(customDimensions.alObjectId)
| where alObjectId between (50000 .. 99999) or alObjectId between (1000000 .. 74999999)
| extend codeOwnership = case(
      alObjectId between (50000 .. 99999), 'ENVIRONMENT'
    , alObjectId between (1000000 .. 69999999), 'ISV_ONPREM'
    , alObjectId between (70000000 .. 74999999), 'ISV_ONLINE'
    , 'MICROSOFT'
)
| project numberOfRows = toint(customDimensions.numberOfRows)
, sqlRowsRead = toint( customDimensions.sqlRowsRead )
, sqlExecutes = toint( customDimensions.sqlExecutes )
, alObjectName = tostring( customDimensions.alObjectName )
, alObjectId = tostring( customDimensions.alObjectId )
, codeOwnership
, extensionPublisher = tostring(customDimensions.extensionPublisher )
| where isnotnull( sqlExecutes ) and isnotnull( sqlRowsRead )
| summarize numberOfReportExecutions=count(), totalRows=sum(numberOfRows), totalSQLExecutes = sum(sqlExecutes), totalSQLRowsRead = sum(sqlRowsRead) 
by alObjectId, alObjectName, codeOwnership
| extend sqlratio = toreal(totalSQLRowsRead)/totalSQLExecutes
, datasetratio = toreal(totalRows)/totalSQLExecutes
| order by sqlratio asc
| limit 10

## Slow AL code (only available from version 17.1)

Learn more about how to write performant AL code here in the performance tuning guide: [https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/performance/performance-developer#al-performance-patterns](https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/performance/performance-developer#al-performance-patterns)

Long running AL method telemetry docs: [https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/administration/telemetry-al-method-trace](https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/administration/telemetry-al-method-trace)

KQL samples:

-   [https://github.com/microsoft/BCTech/blob/master/samples/AppInsights/KQL/Queries/RawData/LongRunningAL.kq](https://github.com/microsoft/BCTech/blob/master/samples/AppInsights/KQL/Queries/RawData/LongRunningAL.kql)l
-   [https://github.com/microsoft/BCTech/blob/master/samples/AppInsights/KQL/Queries/PerformanceTuning/SlowALMethods.kql](https://github.com/microsoft/BCTech/blob/master/samples/AppInsights/KQL/Queries/PerformanceTuning/SlowALMethods.kql)

In [42]:
%%kql
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
let _startDate = startDate;
let _endDate = endDate;
traces
| where 1==1 
    and timestamp >= todatetime(_startDate)
    and timestamp <= todatetime(_endDate) + totimespan(24h) - totimespan(1ms)   
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.environmentName == _environmentName
    and customDimensions.eventId == 'RT0018'
| extend alObjectId = toint(customDimensions.alObjectId)
| where alObjectId between (50000 .. 99999) or alObjectId between (1000000 .. 74999999)
| extend codeOwnership = case(
      alObjectId between (50000 .. 99999), 'ENVIRONMENT'
    , alObjectId between (1000000 .. 69999999), 'ISV_ONPREM'
    , alObjectId between (70000000 .. 74999999), 'ISV_ONLINE'
    , 'MICROSOFT'
)    
| extend clientType = tostring( customDimensions.clientType )
| summarize count=count() by clientType, bin(timestamp, 1d)
| render timechart title= 'Number of long running AL methods (shown by client/session type)'

In [43]:
%%kql
// Top 20 slowest AL methods
//
// 
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
let _startDate = startDate;
let _endDate = endDate;
traces
| where 1==1 
    and timestamp >= todatetime(_startDate)
    and timestamp <= todatetime(_endDate) + totimespan(24h) - totimespan(1ms)   
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.environmentName == _environmentName
    and customDimensions.eventId == 'RT0018'
| extend alObjectId = toint(customDimensions.alObjectId)
| where alObjectId between (50000 .. 99999) or alObjectId between (1000000 .. 74999999)
| extend codeOwnership = case(
      alObjectId between (50000 .. 99999), 'ENVIRONMENT'
    , alObjectId between (1000000 .. 69999999), 'ISV_ONPREM'
    , alObjectId between (70000000 .. 74999999), 'ISV_ONLINE'
    , 'MICROSOFT'
)    
| extend 
  alMethod = tostring( customDimensions.alMethod )
, alObjectName = tostring( customDimensions.alObjectName )
, alObjectType = tostring(customDimensions.alObjectType)
, executionTime = customDimensions.executionTime
, executionTimeInMS = toreal(totimespan(customDimensions.executionTime))/10000 //the datatype for executionTime is timespan 
, extensionName = tostring( customDimensions.extensionName )
| extend executionTimeInSec = round(executionTimeInMS / 1000, 2)
| summarize count(), avg( executionTimeInSec ), min( executionTimeInSec ), max( executionTimeInSec ), percentile(executionTimeInSec, 95) by alMethod, alObjectType, alObjectId, alObjectName, extensionName
| order by avg_executionTimeInSec desc 
| limit 20

## Page views (only available in BC Online (SaaS))

Page view telemetry docs: [https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/administration/telemetry-page-view-trace](https://docs.microsoft.com/en-us/dynamics365/business-central/dev-itpro/administration/telemetry-page-view-trace)

KQL samples: [https://github.com/microsoft/BCTech/blob/master/samples/AppInsights/KQL/RawData/PageViews.kql](https://github.com/microsoft/BCTech/blob/master/samples/AppInsights/KQL/RawData/PageViews.kql)

In [44]:
%%kql
//
// Top 10 longest page times (partner-added pages)
// 
let _aadTenantId = aadTenantId;
let _environmentName = environmentName;
let _startDate = startDate;
let _endDate = endDate;
pageViews
| where 1==1 
    and timestamp >= todatetime(_startDate)
    and timestamp <= todatetime(_endDate) + totimespan(24h) - totimespan(1ms)   
    and customDimensions.aadTenantId == _aadTenantId
    and customDimensions.environmentName == _environmentName
| extend alObjectId = toint(customDimensions.alObjectId)
| where alObjectId between (50000 .. 99999) or alObjectId between (1000000 .. 74999999)
| extend objectId = tostring(customDimensions.alObjectId)
| extend codeOwnership = case(
      alObjectId between (50000 .. 99999), 'ENVIRONMENT'
    , alObjectId between (1000000 .. 69999999), 'ISV_ONPREM'
    , alObjectId between (70000000 .. 74999999), 'ISV_ONLINE'
    , 'MICROSOFT'
)
| summarize median_load_time_in_MS = percentile(duration,50) by pageName=name, objectId
| order by median_load_time_in_MS desc
| limit 10